In [8]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai
from autogen import GroupChat, GroupChatManager
from autogen import AssistantAgent
from autogen import UserProxyAgent


llm_config = {
    "seed": 44,  # for caching the response
    "config_list": azureopenai.config_list,
    "temperature": 0  # for coding it is suggested to be less creative
}

assistant = AssistantAgent(
    "inner-assistant",
    description="use yfinance to get the stock data and then use pandas to filter the data",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = UserProxyAgent(
    "inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)


from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

task = "On which days in current year  Microsoft Stock was higher than $370?"

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(society_of_mind_agent, message=task)

user_proxy (to society_of_mind):

On which days in current year  Microsoft Stock was higher than $370?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
society_of_mind (to chat_manager):

On which days in current year  Microsoft Stock was higher than $370?

--------------------------------------------------------------------------------

Next speaker: inner-assistant

inner-assistant (to chat_manager):

To determine the days in the current year when Microsoft stock was higher than $370, we need to follow these steps:

1. **Fetch Historical Stock Data**: We need to get the historical stock data for Microsoft (MSFT) for the current year.
2. **Filter Data**: Filter the data to find the days when the stock price was higher than $370.
3. **Output the Dates**: Print the dates when the stock price was higher than $370.

We can use the `yfinance` library in Python to fetch the historical stock data. Here is the complete code to ach

ChatResult(chat_id=None, chat_history=[{'content': 'On which days in current year  Microsoft Stock was higher than $370?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Microsoft stock was higher than $370 on the following days in the current year:\n\n- January 8, 9, 10, 11, 12, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31\n- February 1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 20, 21, 22, 23, 26, 27, 28, 29\n- March 1, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 25, 26, 27, 28\n- April 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30\n- May 1, 2, 3, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 28, 29, 30, 31\n- June 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 17, 18, 20, 21, 24, 25, 26, 27, 28\n- July 1, 2, 3, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31\n- August 1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 26, 27, 28, 29', 'role': 'user', 'name': 'society_of_mind'}], summary='Microsoft s